In [1]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [2]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2b-it-q4_k.gguf", device_map="tpu:0",
                                         from_type="gemma",
                                        #  load_eager=True
                                         )

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("NousResearch/gemma-2b-it-tokenizer")
tokenizer.padding_side = "right"
format_prompt = """<start_of_turn>user\n
{}\n
<start_of_turn>model\n
{}"""

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [32]:
from micrlhf.sampling import sample
sample(llama, tokenizer, format_prompt.format("Hi!", ""),
       batch_size=4, do_sample=True)[0]

128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128


LoweringException: Exception while lowering eqn:
  a:bf16[16,256,128] = transpose[permutation=(0, 2, 1)] b
With context:
  LoweringRuleContext(lowering_context=LoweringContext(ir_context=<jaxlib.mlir._mlir_libs._site_initialize.<locals>.Context object at 0x7ed75ba45490>, grid_rank=2, mapped_dims=(), user_grid_indices=(<jaxlib.mlir._mlir_libs._mlir.ir.BlockArgument object at 0x7ed7592db730>, <jaxlib.mlir._mlir_libs._mlir.ir.BlockArgument object at 0x7ed758cab530>), block_shapes=[(8, 128, 12), (8, 128, 128), (8, 1, 128), (8, 1, 128), (128, 4096), (128, 128), None], name_stack=NameStack(stack=()), mesh_context=None, traceback_caches=TracebackCaches(traceback_cache={<jaxlib.xla_extension.Traceback object at 0x563fcb8c40e0>: loc(callsite("matmul_4bit_kernel"("/home/neverix/micrlhf-progress/micrlhf/quantizers.py":128:21) at callsite("matmul_fast.<locals>.kernel_call"("/home/neverix/micrlhf-progress/micrlhf/quantizers.py":274:18) at callsite("matmul_fast"("/home/neverix/micrlhf-progress/micrl